Ameoba CNN for the final project

In [1]:
#save Ram from exploding
#Begone memory
def memory():
    def Begone_memory(exceptions=None):
        if exceptions is None:
            exceptions = []

        for var in list(globals().keys()):
            if var not in exceptions:
                del globals()[var]
    Begone_memory()
    def Begone_memory(exceptions=None):
        if exceptions is None:
            exceptions = []

        for var in list(globals().keys()):
            if var not in exceptions:
                del globals()[var]
memory()
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from openpyxl.styles.builtins import output
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from itertools import product
from collections import OrderedDict, namedtuple
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import shutil
import RunManager as rm
import RunBuilder as bm

# Utility Functions
def imshow(img, text=None, should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic', fontweight='bold', bbox={'facecolor': 'white', 'alpha': 0.8, 'pad': 10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def show_plot(iteration, loss):
    plt.plot(iteration, loss)
    plt.show()

def sanitize_for_windows_path(name):
    # Replace invalid characters with underscores or remove them
    invalid_chars = r'<>:"/\\|?*'
    return ''.join('_' if c in invalid_chars else c for c in name)


In [2]:
img1_directory = "A:\\3rd_Year_Project\\Project_code\\data\\Siamese_dataset\\img1.npy"
training_directory = "A:\\3rd_Year_Project\\Project_code\\data\\Siamese_dataset\\15.npz"
testing_directory = "A:\\3rd_Year_Project\\Project_code\\data\\Siamese_dataset\\2nd.npz"

In [3]:
img1 = np.load(img1_directory)
print(img1.shape)

img2m = np.load(training_directory)
img2 = img2m["images"]
labels = img2m["labels"]
print(img2.shape)
print(np.mean(labels))

(9244, 128, 128)
(12619, 128, 128)
0.47634519375544815


In [4]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to 100x100 pixels # no we want 128x128 idiot, we like 128x128
    transforms.ToTensor(),           # Convert to Tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1] # this does who knows what, probably jsut set's it up correctly
])

In [5]:
class SiameseNetworkDataset():
    def __init__(self, npz_file=None, transform=None): #load dataset from the provided .npz files
        self.data = np.load(npz_file) #Extract images form loaded data
        self.images = self.data['images'] #Extract labels from the loaded data
        self.labels = self.data['labels'] #Extract labels form the loaded data
        self.transform = transform #stroe any image transofmation(in .transform)

    def __getitem__(self, index):
        img = self.images[index]
        label = self.labels[index] # load the images with their associated label

        img = Image.fromarray(img) # convert from numpy array to PIL image

        #if self.transform: # apply trransofmation if one exists(one does)
        #    img = self.transform(img)

        return img, torch.tensor(label, dtype=torch.float32) # return the transformed image and label as a float tensor

    def __len__(self):
        return len(self.images) #Return the total umber of images in the dataset

In [6]:
class SiameseNetworkDataset(torch.utils.data.Dataset): #
    def __init__(self, img1_dir, img2_dir, transform=None): # preparing the data from the directories
        self.img1_data = np.load(img1_dir)
        self.img1_images = self.img1_data #loading in all the img1 images

        self.img2_data = np.load(img2_dir) #loads the second image data
        self.img2_images = self.img2_data['images'] #loads the images
        self.img2_labels = self.img2_data['labels'] # loads the labels for img2

        self.transform = transform # does some funky transformations

    def __getitem__(self, index):
        #sample a randomly healthy image from img1 (this is easy as the label is always 0 for healthy)
        random_index = np.random.choice(len(self.img1_images)) # randomly loading an inividual image from img1
        img1 = Image.fromarray(self.img1_images[random_index]).convert("L") #opens the image as a greyscale(l stands for luminescence or however you spell it)

        #time to load img2 and label directly from npz file(either healthy or a guy who visits maccies every day *cough* Niamh)
        img2 = self.img2_images[index] # load the image numebred "index"
        label = self.img2_labels[index] # loading the labels according to the image number/index thiongy

        img2 = Image.fromarray(img2).convert("L")  # make img 2 greyscale with some funky stuff I've never seen before but apparently it works
        if self.transform:
            img1 = self.transform(img1)#transform img 1
            img2 = self.transform(img2) # transform img 2

        label = torch.tensor(float(label), dtype=torch.float32) # load label of img2 as a
        return img1, img2, label # finish this tomfoolery and output this stuff
    def __len__(self):#this is supposed to do something, I dont know
        return len(self.img2_images)# ok, now I am even more confused, but the code says it works so lets keep it

In [7]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        # Setting up the Sequential of CNN Layers
        self.cnn1 = nn.Sequential(

            nn.Conv2d(1 # input channels (1 for greyscale)
                      , 96 # output channels
                      , kernel_size=11,stride=1), #
            nn.ReLU(inplace=True), # acticate the function
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2), # normalisation to stabalize training
            nn.MaxPool2d(3, stride=2), # pooling to reduce spacial dimensions? whatever that means

            nn.Conv2d(96, 256, kernel_size=5,stride=1,padding=2), # 2nd conv layer
            nn.ReLU(inplace=True), # activate the layer
            nn.LocalResponseNorm(5,alpha=0.0001,beta=0.75,k=2), # normalisation
            nn.MaxPool2d(3, stride=2), # pooling
            nn.Dropout2d(p=0.3), # dropout to reduceoverfitting

            nn.Conv2d(256,384 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384,256 , kernel_size=3,stride=1,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, stride=2),
            nn.Dropout2d(p=0.3), # all this stuff is the same as before

        )

        # Defining the fully connected layers
        self.fc1 = nn.Sequential(
            nn.Linear(43264, 1024), # input size depends onthe flattened CNN output
            nn.ReLU(inplace=True), # activate the funciton
            nn.Dropout(p=0.5), # dropout for rgulaisation

            nn.Linear(1024, 128), # "linearise" it again
            nn.ReLU(inplace=True), # activate it

            nn.Linear(128, 1)) # originally the was the under bit of this line
                      #,2))
                            #last nn.linear is to get a distance similarity whatever that means

    def forward(self, img1, img2):
        x1 = self.cnn1(img1) # pass both images through the first cnn layer
        x2 = self.cnn1(img2)

        x1 = x1.view(x1.size(0), -1) # flattten the outputs to a 2d tensor
        x2 = x2.view(x2.size(0), -1)

        distance = torch.abs(x1 - x2) # find the absolute difference between the 2 image results
        score = self.fc1(distance) # put the distance into the fc layer to find a similarity score
        output = torch.sigmoid(score)  # make the output either 0, or 1
        return output

In [8]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self):
        super(ContrastiveLoss, self).__init__() # margin defines minimuin distance for dissimilar pairs
        self.bce = nn.BCELoss() # if simimilar pair in the marging hten loss is incurred

    def forward(self, output, label):
        loss_contrastive = self.bce(output, label)
        return loss_contrastive

In [32]:
#Hyper Parameter search with RunManager
params = OrderedDict( # params for the server
    lr=[0.0001, 0.001, 0.0025, 0.005, 0.0075, 0.01, 0.1],
    batch_size=[8, 16, 32, 64, 128, 256],
    number_epochs=[20],
    op=[torch.optim.AdamW, torch.optim.Adam, torch.optim.SGD,torch.optim.RAdam, torch.optim.NAdam])  # Only store the optimizer class here

model = SiameseNetwork()

m = rm.RunManager()
b = bm.RunBuilder.get_runs(params)

for run in b:
    if run.op == torch.optim.SGD:
        optimizer = run.op(model.parameters(), lr=run.lr, weight_decay=0.0005)
    else:
        optimizer = run.op(model.parameters(), lr=run.lr, eps=1e-8, weight_decay=0.0005)

for run in b:
    print(run)

Run(lr=0.0001, batch_size=8, number_epochs=20, op=<class 'torch.optim.adamw.AdamW'>)
Run(lr=0.0001, batch_size=8, number_epochs=20, op=<class 'torch.optim.adam.Adam'>)
Run(lr=0.0001, batch_size=8, number_epochs=20, op=<class 'torch.optim.sgd.SGD'>)
Run(lr=0.0001, batch_size=8, number_epochs=20, op=<class 'torch.optim.radam.RAdam'>)
Run(lr=0.0001, batch_size=8, number_epochs=20, op=<class 'torch.optim.nadam.NAdam'>)
Run(lr=0.0001, batch_size=16, number_epochs=20, op=<class 'torch.optim.adamw.AdamW'>)
Run(lr=0.0001, batch_size=16, number_epochs=20, op=<class 'torch.optim.adam.Adam'>)
Run(lr=0.0001, batch_size=16, number_epochs=20, op=<class 'torch.optim.sgd.SGD'>)
Run(lr=0.0001, batch_size=16, number_epochs=20, op=<class 'torch.optim.radam.RAdam'>)
Run(lr=0.0001, batch_size=16, number_epochs=20, op=<class 'torch.optim.nadam.NAdam'>)
Run(lr=0.0001, batch_size=32, number_epochs=20, op=<class 'torch.optim.adamw.AdamW'>)
Run(lr=0.0001, batch_size=32, number_epochs=20, op=<class 'torch.optim.

In [26]:
# Define validation dataset
val_dataset = SiameseNetworkDataset(img1_directory, #Directory containing the first set of images(anchor images)
                                    testing_directory, # derecotry containing hte second set of images(comparision images)
                                    transform=transforms.Compose([ #Defie image transformations
                                        transforms.Resize((128, 128)), # resize to 128x128 pixels
                                        transforms.ToTensor() # convert images to pytorch tensors
                                    ]))

# Create DataLoader using val_dataset
validation_dataloader = DataLoader(val_dataset, # dataset to lead from (defined above)
                                   shuffle=False, # No need to shefful for validation
                                   num_workers=0, # Number of parralel processes(1 thread)
                                   batch_size=run.batch_size, # Number of samples per batch, defined in training config
                                   pin_memory=True) # pin memory to speed up data transfer to gpu (reccommended when using Cuda)

In [27]:
# Load the dataset from raw image folders

siamese_dataset = SiameseNetworkDataset(
    img1_dir=r"/data/Siamese_dataset/img1.npy", #Directory containing the first set of images(anchor images)
    img2_dir=training_directory, # derecotry containing hte second set of images(comparision images)
    transform=transforms.Compose([ #Defie image transformations
        transforms.Resize((128, 128)), # resize to 128x128 pixels
        transforms.ToTensor()])) # convert images to pytorch tensors


train_dataloader = DataLoader(siamese_dataset, # loading inthe siamese dataset defined above
                              shuffle=True, #shuffle for training to randomise things
                              num_workers=0, # no. parralllel processes
                              batch_size=run.batch_size, # set a batch size
                              pin_memory= True) # pinmemory to speed up data transfer to the gpu

In [28]:
#Hyperparameter Search with RunManager and Run Builder
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run', params.keys())
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs


class RunManager():
    def save_checkpoint(self, model, optimizer, epoch, filename="checkpoint.pth"):
        """Save model checkpoint at the current epoch."""
        print(f"Saving checkpoint at epoch {epoch}")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, filename)

    def validate(self, model, validation_dataloader):
        model.eval()  # Switch to evaluation mode
        correct = 0
        total = 0
        loss_history = []

        with torch.no_grad():  # Disable gradient calculation for validation
            for data in validation_dataloader:
                img1, img2, label = data
                img1, img2, label = img1.cuda(), img2.cuda(), label.cuda()

                output = model(img1, img2).squeeze(1)
                loss_contrastive = criterion(output, label)
                loss_history.append(loss_contrastive.item())

                # Compute accuracy#
                #euclidean_distance = F.pairwise_distance(output)
                predictions = (output < 0.5).float()
                correct += (predictions == label).sum().item()
                total += label.size(0)

        accuracy = correct / total
        avg_loss = sum(loss_history) / len(loss_history)
        print(f"Validation Accuracy: {accuracy*100:.4f}, Avg. Validation Loss: {avg_loss:.4f}")

        return accuracy, avg_loss

In [29]:
#def train():
def train(net, train_loader, criterion, optimizer, device, number_epochs = run.number_epochs):
    total_batches_seen = 1
    total_batches = len(train_dataloader) * number_epochs
    start_train = time.time()

    net.to(device) # movethe model to the device

    #for final graph
    epoch_number = 1
    counter = []
    loss_history = []
    epoch_acc = []

    optimizer = run.op(net.parameters(), lr=run.lr) # initialise the optimizer from the run object

    for epoch in range(number_epochs): # loop over all epochs
        total_samples = 0

        net.train() # set model to training mode
        running_loss = 0.0
        correct = 0

        for batch_idx,(img1, img2, label) in enumerate(train_dataloader): # get a pair of images and their similatiry, 0 for same, 1 for differnet, label is the label for img2
            current_iter = epoch * len(train_dataloader) + batch_idx + 1
            total_iter = len(train_dataloader) * number_epochs

            img1, img2, label = img1.cuda(), img2.cuda(), label.cuda() # load to gpu

            optimizer.zero_grad() # resets gradients for this batch
            output = net(img1, img2).squeeze(1) # output should be similarity score (ideally 0 = similar, 1 = different)
            loss_contrastive = criterion(output, label) # find loss between predicted similarity score and true label
            loss_contrastive.backward()  #backpropogate the loss to compute the gradients
            optimizer.step() # update modeel parameters using computated gradients

            running_loss += loss_contrastive.item() # track the running loss

            end_batch = time.time()

            # Calculate accuracy
            predicted = (output < 0.5).float()  # similarity scores < 0.5 = similar (label 0)
            correct += (predicted == label).sum().item()  # count how many predictions matched the label
            accuracy1 = correct / total_batches_seen

            total_samples += label.size(0)

            if total_batches_seen % 10000 == 0 or total_batches_seen in{10} :
                elapsed_time = end_batch - start_train
                mean_time_per_batch = elapsed_time / total_batches_seen
                batches_left = total_batches - (batch_idx + 1 + epoch * len(train_dataloader))

                progress_fraction =current_iter / total_iter

                remaining_time = batches_left * mean_time_per_batch
                hours = int(remaining_time // 3600)
                minutes = int((remaining_time % 3600) // 60)
                seconds = int(remaining_time % 60)

                print(f"We are {progress_fraction * 100:.2f}% complete. ETA: {hours}h {minutes}m {seconds}s")
            total_batches_seen += 1

        # Compute and store accuracy for the epoch
        epoch_acc.append(accuracy1 * 100)
        counter.append(epoch_number)
        epoch_number += 1#add one to the current value of epoch_number
        loss_history.append(running_loss)

    plt.plot(counter, loss_history)
    plt.show()
    return net, counter, loss_history, epoch_acc #return trained model after all epochs are complete

In [30]:
#imitialiseparameters to trackbest model and corresponding optimiser
best_validation_accuracy = 0.0#track highest validation accuracy seen
best_model = None #stores best perfoming model
best_optimizer = None # stores optimiser for said model

m = RunManager()

net = SiameseNetwork().cuda() # create fresh instance fo the Siamese netowrk and move to gpu
criterion = ContrastiveLoss() # set up loss function(stays same for all runs

#loop through all hyperparameter combinations
for run in b:
    start_whole_run = time.time()
    print(f"Running hyperparameter set: {run}") # print the hyperparameters being used

    # Create DataLoader with the specified batch size
    train_dataloader = torch.utils.data.DataLoader(siamese_dataset, batch_size=run.batch_size, shuffle=True) #create the training set usingthe hyperparameters
    validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=run.batch_size, shuffle=False)#create calidation set for curent hyperparameters

    # Initialize model and optimizer
    model = SiameseNetwork()
    model = model.cuda()  # Move to GPU if available
    optimizer = run.op(model.parameters(), lr=run.lr)

    # Train the model
    model, counter, loss_history, epoch_acc = train(model, train_dataloader, criterion, optimizer, device = torch.device("cuda:0"))

    # Validate the model
    accuracy, avg_loss = m.validate(model, validation_dataloader) # should return the accuracy and average loss for the validation data

    # Save the best model
    if accuracy > best_validation_accuracy: # if current model better than previous model then append to
        best_validation_accuracy = accuracy
        best_model = model
        best_optimizer = optimizer
        filename = f"best_model_run_lr{run.lr}_bs{run.batch_size}_epochs{run.number_epochs}.pth"
        m.save_checkpoint(model, optimizer, run.number_epochs, filename=filename) # save the best performing models checkpoint

    end_whole_run = time.time()

print(f"Best Validation Accuracy: {best_validation_accuracy:.4f}")

Running hyperparameter set: Run(lr=0.001, batch_size=16, number_epochs=50, op=<class 'torch.optim.adamw.AdamW'>)
We are 0.03% complete. ETA: 11h 34m 17s


KeyboardInterrupt: 